This script:
- Computes omega column based on the subweights for homogenous timestep TDR results
- Gnerates a heterogenous timestep system TDR input data: 44 x 1h & 22 x 2h
- Computes Rel_TimeStep for the heterogenous timesteps to keep track of how many hours are behind each timestep

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import math

## Demand_data.csv "omega" changes

In [2]:
path = os.getcwd()
p_demand = "Demand_data.csv"
path_demand = os.path.join(path, p_demand)
df_demand = pd.read_csv(path_demand)
df_demand['omega'] = 1
df_demand.to_csv(p_demand)

## Demand_data.csv

In [3]:
path = os.getcwd()
p_demand = "Demand_data.csv"
p_fuel = "Fuels_data.csv"
p_CF = "Generators_variability.csv"

path_demand = os.path.join(path, p_demand)
path_fuel = os.path.join(path, p_fuel)
path_CF = os.path.join(path, p_CF)

df_demand = pd.read_csv(path_demand)
df_fuel = pd.read_csv(path_fuel)
df_CF = pd.read_csv(path_CF)

In [4]:
def get_group_number(row_number, rows_per_group=8):
    if row_number == 0:
        return 0
    return (row_number - 1) // rows_per_group

timesteps_per_rep_period = df_demand.at[0, 'Timesteps_per_Rep_Period']
df_demand['omega'] = [df_demand.at[get_group_number(i), 'Sub_Weights']/timesteps_per_rep_period for i in range(1, len(df_demand) + 1)]

In [5]:
midpoint = len(df_demand) // 2  # This would be 44 for 88 rows
modified_demand = df_demand.copy()
modified_demand['Rel_TimeStep'] = 1
avg_columns = ['Demand_MW_z1', 'Demand_MW_z2', 'Demand_MW_z3']

# Initialize lists to store the results
rows_to_keep = list(range(midpoint))  # Keep all rows in first half

# For the second half, keep every other row
for i in range(midpoint, len(df_demand), 2):
    if i + 1 < len(df_demand):  # Make sure we have a next row
        
        modified_demand.loc[i+1, :] = modified_demand.loc[i, :]

# Keep only the selected rows
modified_demand['Time_Index'] = range(1, len(modified_demand)+1)

# Verify the results
print(f"Original number of rows: {len(modified_demand)}")
print(f"Final number of rows: {len(modified_demand)}")

Original number of rows: 88
Final number of rows: 88


## Fuels_data.csv

In [6]:
midpoint = (len(df_fuel) // 2)+1  # This would be 44 for 88 rows
modified_fuel = df_fuel.copy()

# Initialize lists to store the results
rows_to_keep = list(range(midpoint))  # Keep all rows in first half

# For the second half, keep every other row
for i in range(midpoint, len(df_fuel), 2):
    if i + 1 < len(df_fuel):  # Make sure we have a next row
        
        modified_fuel.loc[i+1, :] = modified_fuel.loc[i, :]
       
# Keep only the selected rows
modified_fuel['Time_Index'] = range(0, len(modified_fuel))

modified_fuel

,Time_Index,CT_NG,None
0,0,0.05306,0.0
1,1,5.45000,0.0
2,2,5.45000,0.0
3,3,5.45000,0.0
4,4,5.45000,0.0
...,...,...,...
84,84,5.45000,0.0
85,85,5.45000,0.0
86,86,5.45000,0.0
87,87,5.45000,0.0


## Generators_variability

In [7]:
midpoint = (len(df_CF) // 2)  # This would be 44 for 88 rows
modified_CF = df_CF.copy()

# Initialize lists to store the results
rows_to_keep = list(range(midpoint))  # Keep all rows in first half

# For the second half, keep every other row
for i in range(midpoint, len(df_CF), 2):
    if i + 1 < len(df_CF):  # Make sure we have a next row

        modified_CF.loc[i+1, :] = modified_CF.loc[i, :]
       
# Keep only the selected rows
modified_CF['Time_Index'] = range(1, len(modified_CF)+1)

modified_CF

,Time_Index,CT_natural_gas_combined_cycle,CT_onshore_wind,CT_solar_pv,CT_battery,CT_battery_for_electrolyzer,CT_electrolyzer
0,1,1.0,0.569945,0.0,1.0,1.0,1.0
1,2,1.0,0.623259,0.0,1.0,1.0,1.0
2,3,1.0,0.694188,0.0,1.0,1.0,1.0
3,4,1.0,0.647400,0.0,1.0,1.0,1.0
4,5,1.0,0.381774,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
83,84,1.0,0.840292,0.0,1.0,1.0,1.0
84,85,1.0,0.643544,0.0,1.0,1.0,1.0
85,86,1.0,0.643544,0.0,1.0,1.0,1.0
86,87,1.0,0.903208,0.0,1.0,1.0,1.0


## Overwrite TDR results

In [8]:
modified_demand.head(4)

,Unnamed: 0,Column1,Voll,Demand_Segment,Cost_of_Demand_Curtailment_per_MW,Max_Demand_Curtailment,$/MWh,Rep_Periods,Timesteps_per_Rep_Period,Sub_Weights,omega,Time_Index,Demand_MW_z1,Rel_TimeStep
0,0,0,50000.0,1.0,1.0,1.0,2000.0,11.0,8.0,8.0,1.0,1,7850.0,1
1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1.0,2,7424.0,1
2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,1.0,3,7107.0,1
3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1.0,4,6947.0,1


In [9]:
modified_CF.head(4)

,Time_Index,CT_natural_gas_combined_cycle,CT_onshore_wind,CT_solar_pv,CT_battery,CT_battery_for_electrolyzer,CT_electrolyzer
0,1,1.0,0.569945,0.0,1.0,1.0,1.0
1,2,1.0,0.623259,0.0,1.0,1.0,1.0
2,3,1.0,0.694188,0.0,1.0,1.0,1.0
3,4,1.0,0.647400,0.0,1.0,1.0,1.0


In [10]:
modified_fuel.head(4)

,Time_Index,CT_NG,None
0,0,0.05306,0.0
1,1,5.45000,0.0
2,2,5.45000,0.0
3,3,5.45000,0.0


In [11]:
modified_demand.to_csv(path_demand)
modified_fuel.to_csv(path_fuel)
modified_CF.to_csv(path_CF)